<a href="https://colab.research.google.com/github/Benylaode/model_tfidf/blob/main/Model_Rekomendasi_Buku_(Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from datasets import load_dataset
import pickle


In [5]:

try:
    df = pd.read_csv("books.csv")
    print(type(df))

except FileNotFoundError:
    print("File 'Data_Books.csv' tidak ditemukan. Menggunakan data dummy untuk demonstrasi.")
    data = {
        'title': [
            'The Lord of the Rings', 'The Hobbit', 'Pride and Prejudice',
            'Sense and Sensibility', 'To Kill a Mockingbird', '1984',
            'Animal Farm', 'The Great Gatsby', 'Moby Dick', 'War and Peace'
        ],
        'authors': [
            'J.R.R. Tolkien', 'J.R.R. Tolkien', 'Jane Austen',
            'Jane Austen', 'Harper Lee', 'George Orwell',
            'George Orwell', 'F. Scott Fitzgerald', 'Herman Melville', 'Leo Tolstoy'
        ],
        'categories': [
            'Fantasy', 'Fantasy', 'Romance',
            'Romance', 'Classics', 'Dystopian',
            'Satire', 'Classics', 'Adventure', 'Historical Fiction'
        ]
    }
    df = pd.DataFrame(data)

<class 'pandas.core.frame.DataFrame'>


In [6]:
df.head()

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6810 entries, 0 to 6809
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   isbn13          6810 non-null   int64  
 1   isbn10          6810 non-null   object 
 2   title           6810 non-null   object 
 3   subtitle        2381 non-null   object 
 4   authors         6738 non-null   object 
 5   categories      6711 non-null   object 
 6   thumbnail       6481 non-null   object 
 7   description     6548 non-null   object 
 8   published_year  6804 non-null   float64
 9   average_rating  6767 non-null   float64
 10  num_pages       6767 non-null   float64
 11  ratings_count   6767 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 638.6+ KB


In [9]:
df.drop_duplicates(inplace=True)

In [10]:


print(df.duplicated().sum())

0


In [11]:


print(df.isnull().sum())

isbn13               0
isbn10               0
title                0
subtitle          4429
authors             72
categories          99
thumbnail          329
description        262
published_year       6
average_rating      43
num_pages           43
ratings_count       43
dtype: int64


In [12]:
numeric_cols = ['published_year', 'average_rating', 'num_pages', 'ratings_count']
for col in numeric_cols:
    if col in df.columns:
        df[col].fillna(df[col].mean(), inplace=True)

if 'subtitle' in df.columns:
    df['subtitle'].fillna('', inplace=True)


if 'authors' in df.columns:
    df['authors'].fillna('Unknown Author', inplace=True)


if 'categories' in df.columns:
    df['categories'].fillna('Unknown Category', inplace=True)


if 'thumbnail' in df.columns:
    df['thumbnail'].fillna('', inplace=True)

if 'description' in df.columns:
    df['description'].fillna('', inplace=True)


print("\nSetelah mengisi missing values:")
print(df.isnull().sum())



Setelah mengisi missing values:
isbn13            0
isbn10            0
title             0
subtitle          0
authors           0
categories        0
thumbnail         0
description       0
published_year    0
average_rating    0
num_pages         0
ratings_count     0
dtype: int64


<ipython-input-12-c65f3a378a39>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
<ipython-input-12-c65f3a378a39>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'd

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6810 entries, 0 to 6809
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   isbn13          6810 non-null   int64  
 1   isbn10          6810 non-null   object 
 2   title           6810 non-null   object 
 3   subtitle        6810 non-null   object 
 4   authors         6810 non-null   object 
 5   categories      6810 non-null   object 
 6   thumbnail       6810 non-null   object 
 7   description     6810 non-null   object 
 8   published_year  6810 non-null   float64
 9   average_rating  6810 non-null   float64
 10  num_pages       6810 non-null   float64
 11  ratings_count   6810 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 638.6+ KB


In [14]:
df.fillna('', inplace=True)

df['combined_features'] = (
    df['isbn13'].astype(str) + ' ' +
    df['isbn10'].astype(str) + ' ' +
    df['title'].astype(str) + ' ' +
    df['subtitle'].astype(str) + ' ' +
    df['authors'].astype(str) + ' ' +
    df['categories'].astype(str) + ' ' +
    df['thumbnail'].astype(str) + ' ' +
    df['description'].astype(str) + ' ' +
    df['published_year'].astype(str) + ' ' +
    df['average_rating'].astype(str) + ' ' +
    df['num_pages'].astype(str) + ' ' +
    df['ratings_count'].astype(str)
)

print("\nFitur teks berhasil digabungkan.")
print(df[['title', 'authors', 'published_year', 'num_pages']].head())



Fitur teks berhasil digabungkan.
            title                          authors  published_year  num_pages
0          Gilead               Marilynne Robinson          2004.0      247.0
1    Spider's Web  Charles Osborne;Agatha Christie          2000.0      241.0
2    The One Tree             Stephen R. Donaldson          1982.0      479.0
3  Rage of angels                   Sidney Sheldon          1993.0      512.0
4  The Four Loves              Clive Staples Lewis          2002.0      170.0


In [15]:

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_features'])
print(f"\nUkuran matriks TF-IDF: {tfidf_matrix.shape}")
print("TF-IDF Vectorizer berhasil diterapkan.")


Ukuran matriks TF-IDF: (6810, 5000)
TF-IDF Vectorizer berhasil diterapkan.


In [16]:

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(f"\nUkuran matriks Cosine Similarity: {cosine_sim.shape}")
print("Cosine Similarity berhasil dihitung.")


Ukuran matriks Cosine Similarity: (6810, 6810)
Cosine Similarity berhasil dihitung.


In [17]:
class BookRecommender:
    def __init__(self, dataframe, cosine_sim_matrix, indices):
        self.df = dataframe
        self.cosine_sim = cosine_sim_matrix
        self.indices = indices

    def recommend_books(self, title, num_recommendations=5):
        if title not in self.indices:
            print(f"Buku dengan judul '{title}' tidak ditemukan dalam dataset.")
            return []

        idx = self.indices[title]

        sim_scores = list(enumerate(self.cosine_sim[idx]))

        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        sim_scores = sim_scores[1:num_recommendations+1]

        book_indices = [i[0] for i in sim_scores]

        recommended_titles = self.df['title'].iloc[book_indices].tolist()
        return recommended_titles



In [19]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

recommender = BookRecommender(df, cosine_sim, indices)
print("\nObjek BookRecommender berhasil diinisialisasi.")

book_title_to_recommend = 'The Four Loves'
print(f"\nMencari rekomendasi untuk buku: '{book_title_to_recommend}'")
recommendations = recommender.recommend_books(book_title_to_recommend, num_recommendations=5)

if recommendations:
    print(f"Rekomendasi buku untuk '{book_title_to_recommend}':")
    for i, book in enumerate(recommendations):
        print(f"{i+1}. {book}")
else:
    print("Tidak ada rekomendasi yang ditemukan.")

book_title_to_recommend_not_found = 'The Four Loves'
print(f"\nMencari rekomendasi untuk buku: '{book_title_to_recommend_not_found}'")
recommendations_not_found = recommender.recommend_books(book_title_to_recommend_not_found)


Objek BookRecommender berhasil diinisialisasi.

Mencari rekomendasi untuk buku: 'The Four Loves'
Rekomendasi buku untuk 'The Four Loves':
1. C.S. Lewis
2. The Art of Loving
3. The Problem of Pain
4. The Complete C. S. Lewis Signature Classics
5. Tolkien and C.S. Lewis

Mencari rekomendasi untuk buku: 'The Four Loves'


In [ ]:
with open('tfidf_model.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

with open('cosine_sim.pkl', 'wb') as f:
    pickle.dump(cosine_sim, f)

with open('indices.pkl', 'wb') as f:
    pickle.dump(indices, f)

print("Model TF-IDF, cosine similarity, dan indices berhasil disimpan.")

Model TF-IDF, cosine similarity, dan indices berhasil disimpan.
